In [6]:
import hashlib
import json
from time import time
from collections import OrderedDict

In [7]:
class Printed:
    """A base class"""
    def __repr__(self):
        return str(self.__dict__)

In [12]:
class Block(Printed):
    """When a block is added to the chain..."""
    def __init__(self, index, previousHash, transactions, proof, time=time()):
        self.index = index
        self.previousHash = previousHash
        self.timestamp = time
        self.transactions = transactions
        self.proof = proof

In [13]:
class Transaction(Printed):
    """When a transaction is added to a block"""
    def __init__(self, sender, receiver, amount):
        self.sender = sender
        self.receiver = receiver
        self.amount = amount

    def to_ordered_dict(self):
        """Converts into (hash) dict"""
        return OrderedDict([('sender', self.sender), ('receiver', self.receiver), ('amount', self.amount)])


In [14]:
def hashString(string):
    return hashlib.sha256(string).hexdigest()

def hashBlock(block):
    hashingBlock = block.__dict__.copy()
    hashingBlock['transactions'] = [tx.to_ordered_dict() for tx in hashingBlock['transactions']]
    return hashString(json.dumps(hashingBlock, sort_keys=True).encode())

In [15]:
hashBlock(Block(0,'0',[],0))

'5d062d7768fe039d798166143ce396095218e91006b61a17ec59a48f9ce9aa87'

In [16]:
"""Verification: helper methods.."""
        
        
        
class Verification:
    @staticmethod
    def validProof(transactions, lastHash, proof):
        guess = (str([tx.to_ordered_dict() for tx in transactions]) + str(lastHash) + str(proof)).encode()
        _hash = hashString(guess)
        return _hash[0:2] == '00'
        
    @classmethod
    def verifyChain(cls, blockchain):
        """ Verify all blocks in the chain and return True if proof is valid, and False otherwise."""

        for (index, block) in enumerate(blockchain):
            if index == 0:
                continue
            if block.previousHash != hashBlock(blockchain[index - 1]):
                return False
            if not cls.validProof(block.transactions[:-1], block.previousHash, block.proof):
                print('Invalid proof of work')
                return False
        return True


In [17]:
class Blockchain:
    def __init__(self):
        genesis_block = Block(0, '', [], 0, 0)
        self.chain = [genesis_block]
        self.unconfirmedTransaction = []
        self.REWAED = 10
    @property
    def chain(self):
        return self.__chain[:]

    # The setter
    @chain.setter
    def chain(self, data):
        self.__chain = data
        


    def proofOfWork(self):
        
        """proof of work:  works on 
        for adding the unconfirmed transactions, 
        hashing the previous block and guessing a proof number """
        
        last_block = self.__chain[-1]
        lastHash = hashBlock(last_block)
        proof = 0
        while not Verification.validProof(self.unconfirmedTransaction, lastHash, proof):
            proof += 1
        return proof
    
    @property    
    def unconfirmed(self):
        """A list of unconfirmed transactions."""
        return self.unconfirmedTransaction[:]

    def lastChain(self):
        """last block. """
        if len(self.__chain) < 1:
            return None
        return self.__chain[-1]
    
    def addTransaction(self, sender, receiver, amount=0.9):
        """ Append a new transactions"""
        transaction = Transaction(sender, receiver, amount)
        self.unconfirmedTransaction.append(transaction)

        
    def addBlcok(self):
        """Add a new block and append unconfirmed transactions similar to a mining block"""

        last_block = self.__chain[-1]        
        hashed_block = hashBlock(last_block)
        proof = self.proofOfWork()
        
        reward_transaction = Transaction(
            'MINING', 'receiverAddress', self.REWAED)

        copied_transactions = self.unconfirmedTransaction[:]

        copied_transactions.append(reward_transaction)
        
        block = Block(len(self.__chain), hashed_block,
                      copied_transactions, proof)
        
        block.hash = hashBlock(block)

        self.__chain.append(block)
        self.unconfirmedTransaction = []
        
        return block



In [18]:
myblock = Blockchain()

In [19]:
myblock.chain

[{'index': 0, 'previousHash': '', 'timestamp': 0, 'transactions': [], 'proof': 0}]

In [21]:
myblock.addBlcok()

{'index': 2, 'previousHash': '2109b6a0c6c950b4f414a1d3ad47a0b653a2363bf7affc60fccc6fb62148599f', 'timestamp': 1612439019.420285, 'transactions': [{'sender': 'MINING', 'receiver': 'receiverAddress', 'amount': 10}], 'proof': 114, 'hash': '0fdc3c0043e8a1db2a22c4ace73cfa4bae806363f0cb98eb3dc51fa949f3db09'}

In [22]:
myblock.chain

[{'index': 0, 'previousHash': '', 'timestamp': 0, 'transactions': [], 'proof': 0},
 {'index': 1, 'previousHash': '817c94b3a53cf23a06afceb0cbfcd9c71eac120fbf89a47aba1ae8d170d08999', 'timestamp': 1612439019.420285, 'transactions': [{'sender': 'MINING', 'receiver': 'receiverAddress', 'amount': 10}], 'proof': 91, 'hash': '78538f607a3ff98d74b0a3a0c6d4a004414f660ac4b8ab2cb6c4544bd4d53a01'},
 {'index': 2, 'previousHash': '2109b6a0c6c950b4f414a1d3ad47a0b653a2363bf7affc60fccc6fb62148599f', 'timestamp': 1612439019.420285, 'transactions': [{'sender': 'MINING', 'receiver': 'receiverAddress', 'amount': 10}], 'proof': 114, 'hash': '0fdc3c0043e8a1db2a22c4ace73cfa4bae806363f0cb98eb3dc51fa949f3db09'}]

In [30]:
myblock.addTransaction("anwar", "adam", 2)

In [26]:
myblock.chain

[{'index': 0, 'previousHash': '', 'timestamp': 0, 'transactions': [], 'proof': 0},
 {'index': 1, 'previousHash': '817c94b3a53cf23a06afceb0cbfcd9c71eac120fbf89a47aba1ae8d170d08999', 'timestamp': 1612439019.420285, 'transactions': [{'sender': 'MINING', 'receiver': 'receiverAddress', 'amount': 10}], 'proof': 91, 'hash': '78538f607a3ff98d74b0a3a0c6d4a004414f660ac4b8ab2cb6c4544bd4d53a01'},
 {'index': 2, 'previousHash': '2109b6a0c6c950b4f414a1d3ad47a0b653a2363bf7affc60fccc6fb62148599f', 'timestamp': 1612439019.420285, 'transactions': [{'sender': 'MINING', 'receiver': 'receiverAddress', 'amount': 10}], 'proof': 114, 'hash': '0fdc3c0043e8a1db2a22c4ace73cfa4bae806363f0cb98eb3dc51fa949f3db09'}]

In [31]:
myblock.unconfirmed

[{'sender': 'anwar', 'receiver': 'adam', 'amount': 2},
 {'sender': 'anwar', 'receiver': 'adam', 'amount': 2},
 {'sender': 'anwar', 'receiver': 'adam', 'amount': 2}]

In [33]:
myblock.addBlcok()

{'index': 3, 'previousHash': '9ee84c010506452f53710cc2d91078847d8801b8dd6342b1c37e6d1335f184d8', 'timestamp': 1612439019.420285, 'transactions': [{'sender': 'anwar', 'receiver': 'adam', 'amount': 2}, {'sender': 'anwar', 'receiver': 'adam', 'amount': 2}, {'sender': 'anwar', 'receiver': 'adam', 'amount': 2}, {'sender': 'MINING', 'receiver': 'receiverAddress', 'amount': 10}], 'proof': 350, 'hash': '2cca80890de301e3825f0ae1a40e05f9fd2c723166ba3cbabcee513c43876e33'}

In [35]:
myblock.chain

[{'index': 0, 'previousHash': '', 'timestamp': 0, 'transactions': [], 'proof': 0},
 {'index': 1, 'previousHash': '817c94b3a53cf23a06afceb0cbfcd9c71eac120fbf89a47aba1ae8d170d08999', 'timestamp': 1612439019.420285, 'transactions': [{'sender': 'MINING', 'receiver': 'receiverAddress', 'amount': 10}], 'proof': 91, 'hash': '78538f607a3ff98d74b0a3a0c6d4a004414f660ac4b8ab2cb6c4544bd4d53a01'},
 {'index': 2, 'previousHash': '2109b6a0c6c950b4f414a1d3ad47a0b653a2363bf7affc60fccc6fb62148599f', 'timestamp': 1612439019.420285, 'transactions': [{'sender': 'MINING', 'receiver': 'receiverAddress', 'amount': 10}], 'proof': 114, 'hash': '0fdc3c0043e8a1db2a22c4ace73cfa4bae806363f0cb98eb3dc51fa949f3db09'},
 {'index': 3, 'previousHash': '9ee84c010506452f53710cc2d91078847d8801b8dd6342b1c37e6d1335f184d8', 'timestamp': 1612439019.420285, 'transactions': [{'sender': 'anwar', 'receiver': 'adam', 'amount': 2}, {'sender': 'anwar', 'receiver': 'adam', 'amount': 2}, {'sender': 'anwar', 'receiver': 'adam', 'amount': 2

In [37]:
Verification.verifyChain(myblock.chain)

True